# Лабораторна №9. Тензори. Нейронні мережі та табличні дані.
## Робота з тензорами.

### Імпортуйте необхідні біліотеки.

In [2]:
np.random.seed(42) # запустіть цю клітинку, щоб ваші результати співпадали з тим, що наведено у прикладі.
torch.manual_seed(42)

### Створіть numpy масив який містить 12 випадкових цілих чисел в межах від 0 (включно) до 12 (невключно).

In [3]:
# пишіть код тут.

[ 6  3 10  7  4  6  9  2  6 10 10  7]


### Створіть тензор з масиву, створеного вище.

In [5]:
# пишіть код тут.

tensor([ 6,  3, 10,  7,  4,  6,  9,  2,  6, 10, 10,  7], dtype=torch.int32)


### Змініть тип тензора на int64.

In [7]:
# пишіть код тут.

In [8]:
x = x.long()
print(x.type())

torch.LongTensor


### Змініть розміри тензора на 3x4.

In [9]:
# пишіть код тут.

tensor([[ 6,  3, 10,  7],
        [ 4,  6,  9,  2],
        [ 6, 10, 10,  7]])


### Виведіть значення 4 стовпчика.

In [11]:
# пишіть код тут.

tensor([[7],
        [2],
        [7]])

### Без зміни тензора, поверніть тензор, у якому є квадрати значеннь з основного тензора.

In [13]:
# пишіть код тут.

tensor([[ 36,   9, 100,  49],
        [ 16,  36,  81,   4],
        [ 36, 100, 100,  49]])

### Створіть ще одни тензор, з такими ж елементами, як у основному тензорі, але який можна буде перемножити з основним тензором.
Використайте PyTorch для створення тензора.

In [15]:
# пишіть код тут.

tensor([[ 6, 11,  4],
        [10,  6, 11],
        [ 4,  8, 10],
        [ 5, 10, 10]])


### Знайдіть добуток двох матриць: основної, та створеної на минулому кроці.

In [17]:
# пишіть код тут.

tensor([[141, 234, 227],
        [130, 172, 192],
        [211, 276, 304]])


## Нейромережі.
Використайте один з наборів даних, які вже використовувались у попередніх лабораторних роботах. Проведіть аналіз даних, та розділіть їх на незалежні та цільові змінні. Створіть моделі штучної нейронної мережі, які зможуть прогнозувати цільову змінну використовуючи різні фреймворки. Мережі мають містити як мінімум один прихований рівень.